In [6]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


class SearchRecommendation:
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)
        self.data['processed_name'] = self.data['name'].str.lower()

        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(self.data['processed_name'])

    def search(self, query, top_n=20):
        query_vector = self.vectorizer.transform([query.lower()])
        cosine_similarities = cosine_similarity(query_vector, self.tfidf_matrix).flatten()
        results_df = pd.DataFrame({
            'id': self.data['id'].astype(str),  # Ensure ID is treated as string for consistency
            'name': self.data['name'],
            'cosine_similarity': cosine_similarities
        })
        results_df = results_df.sort_values(by='cosine_similarity', ascending=False)

        # Select only the 'id' and 'name' columns and convert to list of dictionaries
        results_list = results_df[['id', 'name']].head(top_n).to_dict(orient='records')

        return results_list


    def save(self, filepath):
        """Save the model to a file."""
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filepath):
        """Load the model from a file."""
        with open(filepath, 'rb') as f:
            return pickle.load(f)


# Example Usage
if __name__ == "__main__":
    # Initialize and save the model
    search_model = SearchRecommendation(data_path="../test_data.csv")
    search_model.save('D:/RecommendModel/Saved_model/search_recommendation_model.pkl')
    # Load the model from the file

    # Perform a search
    query = "men"
    results = search_model.search(query, top_n=20)
    print("Search query:", query)
    print(results)


Search query: men
[{'id': 'mcl216945', 'name': "Men's Casual Shirt for Men"}, {'id': 'mcl015716', 'name': 'R&B Men T-Shirt'}, {'id': 'mcl185879', 'name': 'Lyriq Casual Shirt for Men|| Shirt for Men|| Men Stylish Shirt || Men Printed Shirt'}, {'id': 'str241036', 'name': 'AL FASCINO Wallet for Men Stylish Purse for Men RFID Wallet for Men Purse for Men Leather Wallets for Men Original Wallet ...'}, {'id': 'msh244701', 'name': "Men's Tan Synthetic Loafer for Men's"}, {'id': 'mcl222437', 'name': 'Lymio Casual Shirt for Men|| Shirt for Men|| Men Stylish Shirt || Men Printed Shirt (Big Pan)'}, {'id': 'str250225', 'name': 'Lymio Casual Shirt for Men|| Shirt for Men|| Men Stylish Shirt || Men Printed Shirt (Squre)'}, {'id': 'mcl009011', 'name': "Men's Trendy Boxer/Shorts for Men/Cotton Boxer for Men (Pack of 3)"}, {'id': 'str199507', 'name': 'Lymio Casual Shirt for Men|| Shirt for Men|| Men Stylish Shirt || Men Printed Shirt (Shed Floral)'}, {'id': 'mcl126235', 'name': 'Lymio Casual Shirt for 

In [ ]:
import pickle

# Save the SearchRecommendation instance
def save_search_recommendation_model(model, filepath="search_recommendation_model.pkl"):
    with open(filepath, "wb") as file:
        pickle.dump(model, file)
    print("SearchRecommendation model saved successfully.")

# Example usage
model = SearchRecommendation(data_path='../test_data.csv')
save_search_recommendation_model(model)
